In [2]:
import pandas as pd
import os

path = os.path.join('..', 'dev', 'db_creation', 'table_making.csv')
df = pd.read_csv(path, index_col=0)

In [3]:
df[['street_number', 'street_name']] = df.qAddress.str.split(' ', n=1, expand=True)
df = df[['street_number', 'street_name', 'qSpecies']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174480 entries, 121399 to 15238
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   street_number  174478 non-null  object
 1   street_name    174478 non-null  object
 2   qSpecies       174480 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 5.3+ MB


In [5]:
df = df.dropna().astype({'street_number': 'int64'}).sort_values(['street_name', 'street_number'])

In [19]:
df.loc[df.street_name == 'san jose ave']

,street_number,street_name,qSpecies
TreeID,,,
39786,50,san jose ave,31
39791,50,san jose ave,35
39780,50,san jose ave,31
39789,50,san jose ave,31
39790,50,san jose ave,275
...,...,...,...
93576,2998,san jose ave,30
93577,2998,san jose ave,80
93574,2998,san jose ave,80


In [38]:
# {street_name: (street_number, qSpecies)}
street = df.loc[df.street_name == 'valencia st'][['street_number', 'qSpecies']]
tree_list = [None] * len(street)
for i, row in enumerate(street.itertuples()):
    street_number = row.street_number
    qSpecies = row.qSpecies
    tree_list[i] = (street_number, qSpecies)

tree_list

[(3, 334),
 (4, 2),
 (4, 2),
 (8, 2),
 (12, 2),
 (12, 2),
 (12, 2),
 (14, 2),
 (14, 2),
 (16, 2),
 (75, 96),
 (75, 96),
 (95, 96),
 (95, 96),
 (98, 2),
 (98, 2),
 (98, 2),
 (98, 2),
 (98, 2),
 (100, 25),
 (101, 26),
 (101, 26),
 (101, 2),
 (101, 26),
 (101, 26),
 (101, 2),
 (101, 2),
 (101, 26),
 (101, 2),
 (101, 26),
 (101, 2),
 (111, 9),
 (115, 9),
 (115, 9),
 (135, 26),
 (135, 99),
 (145, 9),
 (145, 9),
 (147, 9),
 (151, 176),
 (151, 9),
 (155, 9),
 (155, 9),
 (170, 9),
 (170, 9),
 (174, 176),
 (174, 2),
 (175, 9),
 (175, 9),
 (175, 176),
 (175, 9),
 (175, 9),
 (175, 25),
 (175, 9),
 (175, 9),
 (175, 9),
 (175, 9),
 (175, 9),
 (177, 9),
 (177, 9),
 (178, 176),
 (198, 2),
 (199, 2),
 (201, 47),
 (201, 96),
 (201, 47),
 (222, 2),
 (224, 2),
 (224, 2),
 (245, 2),
 (245, 2),
 (245, 2),
 (245, 2),
 (245, 2),
 (250, 166),
 (250, 9),
 (250, 9),
 (250, 9),
 (250, 9),
 (250, 166),
 (250, 166),
 (250, 166),
 (250, 9),
 (250, 166),
 (250, 9),
 (250, 166),
 (250, 2),
 (250, 166),
 (250, 166),
 

Making the tree dict

In [ ]:
street_names = df.street_name.unique().tolist()

tree_dict = dict()

for street_name in street_names:
    trees = df.loc[df.street_name == street_name][['street_number', 'qSpecies']]
    
    tree_list = [None] * len(trees)
    
    for i, row in enumerate(trees.itertuples()):
        street_number = row.street_number
        qSpecies = row.qSpecies
        tree_list[i] = (street_number, qSpecies)
        
    tree_dict.update({street_name: tree_list})

In [43]:
import sys
sys.getsizeof(tree_dict)/1000

73.816

In [131]:
len(tree_dict)

2030

Algorithm to find trees at address or adjacent address (within street_buffer)

In [173]:
%%timeit
street = tree_dict['valencia st']
trees =  list()
street_num = 600
search_nearby = False
street_buffer = 11

def bisect(street: list, street_num: int) -> list:
    length = len(street)
    middle = int(length / 2)
    
    # street buffer incase bisection lands on previous address
    if street[middle][0] < street_num - street_buffer:
         return bisect(street[middle:], street_num)
    elif street[middle][0] > street_num + street_buffer:
        return bisect(street[:middle+1], street_num)
    else:
        return street
    
street = bisect(street, street_num)


for i, address in enumerate(street):
    if address[0] >= street_num:
        if address[0] ==  street_num:
            trees.append(address)
        elif trees:
            break
        elif address[0] > street_num and address[0] < street_num + street_buffer:
            search_nearby = True
            next_address_start = i
            next_address = address[0]
            break

if search_nearby:
    for address in street[next_address_start:]:
        if address[0] == next_address:
            trees.append(address)
        else:
            break
            
    previous_address = street[next_address_start - 1][0]
    if previous_address > street_num - street_buffer: 
        for address in street[next_address_start + 1::-1]:
            if address[0] == previous_address:
                trees.append(address)
            elif address[0] < previous_address:
                break

trees

24.5 µs ± 3.22 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Testing pickling 

In [ ]:
test = pickle.dumps(tree_dict['valencia st'])

In [137]:
%%timeit
pickle.loads(test)

60.4 µs ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [182]:
import pickle
with open('tree_dict_test.pkl', 'wb') as fp:
    pickle.dump(tree_dict, fp)

In [185]:
%%timeit
with open('tree_dict_test.pkl', 'rb') as fp:
    test_dict = pickle.load(fp)

18.5 ms ± 66.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Splitting evens and odds

In [174]:
street_names = df.street_name.unique().tolist()

tree_dict = dict()

for street_name in street_names:
    trees = df.loc[df.street_name == street_name][['street_number', 'qSpecies']]
    
    odd_tree_list = list()
    even_tree_list = list()
    
    for row in trees.itertuples():
        street_number = row.street_number
        qSpecies = row.qSpecies
        if street_number % 2 == 1:
            odd_tree_list.append((street_number, qSpecies))
        else:
            even_tree_list.append((street_number, qSpecies))
        
    tree_dict.update({street_name: {'odd': odd_tree_list, 'even': even_tree_list}})

In [181]:
sys.getsizeof(tree_dict)/1000

73.816

In [186]:
%%timeit
street = tree_dict['valencia st']
trees =  list()
street_num = 600

if street_num % 2 == 1:
    street = street['odd']
else:
    street = street['even']

street_buffer = 11

def bisect(street: list, street_num: int) -> list:
    length = len(street)
    middle = int(length / 2)
    
    # street buffer incase bisection lands on previous address
    if street[middle][0] < street_num - street_buffer:
         return bisect(street[middle:], street_num)
    elif street[middle][0] > street_num + street_buffer:
        return bisect(street[:middle+1], street_num)
    else:
        return street
    
street = bisect(street, street_num)

search_nearby = False


for i, address in enumerate(street):
    if address[0] >= street_num:
        if address[0] ==  street_num:
            trees.append(address)
        elif trees:
            break
        elif address[0] > street_num and address[0] < street_num + street_buffer:
            search_nearby = True
            next_address_start = i
            next_address = address[0]
            break

if search_nearby:
    for address in street[next_address_start:]:
        if address[0] == next_address:
            trees.append(address)
        else:
            break
            
    previous_address = street[next_address_start - 1][0]
    if previous_address > street_num - street_buffer: 
        for address in street[next_address_start + 1::-1]:
            if address[0] == previous_address:
                trees.append(address)
            elif address[0] < previous_address:
                break

trees

6.05 µs ± 326 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Creating pure dict / hash table

In [7]:
df.loc[df.street_name == 'valencia st']

,street_number,street_name,qSpecies
TreeID,,,
23029,3,valencia st,334
228856,4,valencia st,2
22879,4,valencia st,2
22881,8,valencia st,2
22880,12,valencia st,2
...,...,...,...
84888,1699,valencia st,40
84890,1699,valencia st,40
220535,1699,valencia st,213


In [11]:
street_names = df.street_name.unique().tolist()

tree_dict = dict()

for street_name in street_names:
    street_trees = df.loc[df.street_name == street_name][['street_number', 'qSpecies']]
    tree_dict.update({street_name: {}})
    
    for row in street_trees.itertuples():
        
        if row.street_number in tree_dict[street_name]:
            tree_dict[street_name][row.street_number].append(row.qSpecies)
        else:
            tree_dict[street_name].update({row.street_number: [row.qSpecies]})
            

In [20]:
%%timeit
street = 'brotherhood way'
street_num = 900
address_buffer = 11

trees = {}

try:
    trees[street_num] = tree_dict[street][street_num]
except KeyError:
    max_street_num = street_num + address_buffer
    min_street_num = street_num - address_buffer
    original_street_num = street_num
    
    while street_num <= max_street_num:
        street_num = street_num + 2
        try:
            trees[street_num] = tree_dict[street][street_num]
            break
        except KeyError:
            pass
    while street_num >= min_street_num:
        street_num = street_num - 2
        try:
            trees[street_num] = tree_dict[street][street_num]
            break
        except KeyError:
            pass
trees

176 ns ± 2.27 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Make species dict

In [21]:
path = os.path.join('..', 'dev', 'mapped_species.csv')
species_df = pd.read_csv(path, index_col=0)

In [22]:
species_df.head()

,qSpecies,urlPath
0,Corymbia ficifolia :: Red Flowering Gum,540
1,Eucalyptus polyanthemos :: Silver Dollar Eucal...,561
2,Lophostemon confertus :: Brisbane Box,1425
3,Cupressus macrocarpa :: Monterey Cypress,476
4,Jacaranda mimosifolia :: Jacaranda,741


In [25]:
species_dict = species_df.to_dict('index')


In [27]:
sys.getsizeof(species_dict)/1000

18.52